In [1]:
import pandas as pd
import os 
import pathlib 
import glob
from datetime import datetime
from tqdm import tqdm

from dotenv import load_dotenv
from elasticsearch import Elasticsearch

import minio
from minio.error import S3Error
from minio.commonconfig import ENABLED
from minio.versioningconfig import VersioningConfig

from RDSBucket_class import *
from data_profiles import *

import warnings
warnings.filterwarnings("ignore")

#####----------------------------------------------------------------#####
##### preprocessing metadata for bam file
#####----------------------------------------------------------------#####

path_to_save_prep_metadata = "/Users/hieunguyen/src/DVC_system/examples/dummy_from_real/prep_metadata"
os.system("mkdir -p {}".format(path_to_save_prep_metadata))

path_to_main_input = "./examples/dummy_data"    
minio_credential = "credentials.mb.json"
es_credential = "es_credential.json"

donwloaddir = "./examples/download"
os.system("mkdir  -p {}".format(donwloaddir))

##### genearte the connection to Elasticsearch
es = ESearch(es_credential = es_credential)

##### define search query
search_query = {
  "query": {
    "match": {
      "Labcode": "ZLBE113NB"
    }
  }
}

##### define search index
search_indices = "wgsfeature"

##### get the search results
search_res = es.search(search_indices = "wgsfeature", search_query = search_query)


In [6]:
i = 0
bucketName = search_res.loc[i]["bucket"]
object_name = search_res.loc[i]["FileName"]
versionID = search_res.loc[i]["versionID"]

download_selected_file(minio_credential = minio_credential, 
                       bucketName = bucketName,
                       object_name = object_name, 
                       versionID = versionID, 
                       downloaddir = donwloaddir)

File '9-ZLBE113NB_S95053-S97053_GWfeature_CNA_short.rds' downloaded successfully.


True